In [29]:
# Import the dependencies
import pandas as pd
import numpy as np

# Import the requests library
import requests

# Import the API key
from config import g_key

# Import dependencies for directions layer
import gmaps
import gmaps.datasets



In [30]:
# Import WeatherPy_vacation csv and save as a new Dataframe
vacation_it_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_it_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,San Andres,CO,78.80,light intensity drizzle,12.58,-81.70,Hotel Arena Blanca
1,7,Kulhudhuffushi,MV,84.22,overcast clouds,6.62,73.07,Haajy Guest House 1
2,10,Danville,US,77.00,clear sky,36.59,-79.39,Holiday Inn Express Danville
3,11,Opobo,US,77.00,clear sky,36.59,-79.39,Holiday Inn Express Danville
4,17,Rikitea,PF,70.77,clear sky,-23.12,-134.97,Pension Maro'i


In [31]:
# Creating separate DataFrames for four cities within the same continent
bethel_us_df = vacation_it_df.loc[(vacation_it_df["City"] == "Bethel") & \
                                       (vacation_it_df["Country"] == "US")]

st_augustine_us_df = vacation_it_df.loc[(vacation_it_df["City"] == "Saint Augustine") & \
                                       (vacation_it_df["Country"] == "US")]

north_platte_us_df = vacation_it_df.loc[(vacation_it_df["City"] == "North Platte") & \
                                       (vacation_it_df["Country"] == "US")]

westport_us_df = vacation_it_df.loc[(vacation_it_df["City"] == "Westport") & \
                                       (vacation_it_df["Country"] == "US")]

In [32]:
bethel_us_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
74,225,Bethel,US,75.0,clear sky,41.37,-73.41,Courtyard by Marriott Danbury


In [33]:
st_augustine_us_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
105,292,Saint Augustine,US,86.0,clear sky,29.89,-81.31,Best Western Spanish Quarters Inn


In [34]:
north_platte_us_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
160,415,North Platte,US,71.01,clear sky,41.12,-100.77,Holiday Inn Express & Suites North Platte


In [35]:
westport_us_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
202,504,Westport,US,79.0,clear sky,41.14,-73.36,The Westport Inn


In [36]:
# Creating latitude and longitude pairs for each city DataFrame as tuples

lat_lng_pairs_df=pd.DataFrame({"Bethel": [41.37, -73.41], "Saint Augustine": [29.89, -81.31], "North Platte": [41.12, -100.77], "Westport": [41.14, -73.36]})
lat_lng_pairs_df.to_numpy()

array([[  41.37,   29.89,   41.12,   41.14],
       [ -73.41,  -81.31, -100.77,  -73.36]])

In [46]:
# Creating direction layer for Bethel to Saint Augustine
fig = gmaps.figure()
Bethel2StAugustine = gmaps.directions_layer(lat_lng_pairs_df["Bethel"], lat_lng_pairs_df["Saint Augustine"]
                                               ,travel_mode="DRIVING")
StAugustine2Nplatte = gmaps.directions_layer(lat_lng_pairs_df["Saint Augustine"], lat_lng_pairs_df["North Platte"]
                                               ,travel_mode="DRIVING")
Nplatte2Westport = gmaps.directions_layer(lat_lng_pairs_df["North Platte"], lat_lng_pairs_df["Westport"]
                                               ,travel_mode="DRIVING")

fig.add_layer(Bethel2SaintAugustine)
fig.add_layer(StAugustine2Nplatte)
fig.add_layer(Nplatte2Westport)

fig

Figure(layout=FigureLayout(height='420px'))

In [48]:
# Merging 4 separate DataFrames for each city into one
df1=pd.concat([bethel_us_df, st_augustine_us_df, north_platte_us_df, westport_us_df], ignore_index=True)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,225,Bethel,US,75.00,clear sky,41.37,-73.41,Courtyard by Marriott Danbury
1,292,Saint Augustine,US,86.00,clear sky,29.89,-81.31,Best Western Spanish Quarters Inn
2,415,North Platte,US,71.01,clear sky,41.12,-100.77,Holiday Inn Express & Suites North Platte
3,504,Westport,US,79.00,clear sky,41.14,-73.36,The Westport Inn


In [49]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [51]:
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in df1.iterrows()]

In [55]:
# Add the vacation spots and marker for each city
locations = df1[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))